### Setup

First, import the required packages.

Next, specify some important variables:
* `platform_address`: The address of the Yugabyte Platform API
* `platform_api_key`: The API key used to authenticate with the Platform API

Finally, create the Yugabyte Platform API client object.

In [ ]:
import os
import yb_platform_client
from yb_platform_client.api import access_keys_api
from yb_platform_client.api import provider_api
from yb_platform_client.api import session_api
from yb_platform_client.api import universe_cluster_mutations_api
from yb_platform_client.model.universe_configure_task_params \
    import UniverseConfigureTaskParams
from yb_platform_client.model.cluster import Cluster
from yb_platform_client.model.device_info import DeviceInfo
from yb_platform_client.model.provider import Provider
from yb_platform_client.model.user_intent import UserIntent
from pprint import pprint

platform_address = 'http://localhost:9000'
platform_api_key = os.getenv('YB_API_KEY')

api_client = yb_platform_client.ApiClient(yb_platform_client.Configuration(
    host = platform_address,
    api_key = {
        'apiKeyAuth': platform_api_key,
    }
))

### Get Session Info

Make an API call to session endpoint to determine customer UUID.

In [ ]:
session_api = session_api.SessionApi(api_client)

try:
    session_info = session_api.get_session_info()
except yb_platform_client.ApiException as e:
    print("Error get_session_info: %s" % e)
    raise

customer_uuid = session_info.get('customer_uuid')
print('Customer UUID:\n%s' % customer_uuid)

### Get Provider UUID

Make an API call to the provider endpoint to determine provider UUID and regions.

In [ ]:
provider_api = provider_api.ProviderApi(api_client)

try:
    provider_list = provider_api.get_list_of_providers(customer_uuid)
except yb_platform_client.ApiException as e:
    print('Error get_list_of_providers: %s' % e)
    raise

for provider in provider_list:
    if provider.code == 'gcp' and provider.name == 'my-gcp-provider':
        provider_uuid = provider.uuid
        region_list = [region.uuid for region in provider.regions]
        break

print('Provider UUID:\n%s' % provider_uuid)
print('Regions:\n[%s]' % ', '.join(region_list))

### Get Access Key

Make an API call to the access key endpoint to determine access key for provider.

In [ ]:
access_keys_api = access_keys_api.AccessKeysApi(api_client)

try:
    access_key_list = access_keys_api.list(customer_uuid, provider_uuid)
except yb_platform_client.ApiException as e:
    print('Error access key list: %s' % e)
    raise

access_key_code = access_key_list[0]['id_key']['key_code']
print('Access Key:\n%s' % access_key_code)

### Define Universe

In this example, we will create a GCP universe. Define a `Universe` object with the desired configuration.

In [ ]:
new_universe = UniverseConfigureTaskParams(
    clusters=[
        Cluster(
            cluster_type='PRIMARY',
            user_intent=UserIntent(
                universe_name='my-gcp-universe',
                provider_type='gcp',
                provider=provider_uuid,
                region_list=region_list,
                num_nodes=3,
                replication_factor=3,
                instance_type='n1-standard-1',
                device_info=DeviceInfo(
                    num_volumes=1,
                    storage_type='Persistent',
                    volume_size=375,
                ),
                assign_public_ip=True,
                use_time_sync=True,
                enable_ysql=True,
                enable_yedis=False,
                enable_node_to_node_encrypt=True,
                enable_client_to_node_encrypt=True,
                enable_volume_encryption=False,
                yb_software_version='2.7.3.0-b80',
                access_key_code=access_key_code,
                tserver_g_flags={},
                master_g_flags={},
            )
        ),
    ],
)

### Create Universe

Make API call to create new universe.

In [ ]:
universe_api = universe_cluster_mutations_api.UniverseClusterMutationsApi(api_client)

try:
    api_response = universe_api.create_all_clusters(customer_uuid, new_universe)
except yb_platform_client.ApiException as e:
    print('Error create_all_clusters: %s' % e)
    raise

pprint(api_response)